<a href="https://colab.research.google.com/github/Abdall10/colab-1/blob/main/Shakespeare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow

In [2]:
%tensorflow_version 2.x
from keras.datasets import imdb
from keras.models import Sequential
import tensorflow as tf
import os
import numpy as np

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
from google.colab import files
path_to_file= list(files.upload().keys())[0]

Saving bee_movie.md to bee_movie.md


In [11]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print ('Length of text: {} characters'.format(len(text)))

Length of text: 827 characters


In [12]:
vocab = sorted(set(text))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [13]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

In [14]:
seq_length = 100
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences= char_dataset.batch(seq_length+1, drop_remainder=True)

In [15]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)
EMBEDDING_DIM = 256
RNN_UNITS = 1024

BUFFER_SIZE = 10000

data = sequences.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [19]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(batch_shape=(batch_size, None)),  # Explicit input layer
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(
            rnn_units,
            return_sequences=True,
            stateful=True,
            recurrent_initializer='glorot_uniform'
        ),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (64, None, 256)        │        12,032 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (64, None, 1024)       │     5,246,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (64, None, 47)         │        48,175 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,307,183 (20.25 MB)

 Trainable params: 5,307,183 (20.25 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
)

In [22]:
# Directory to save the model checkpoints
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [24]:
data = sequences.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [26]:
# Suppose 'text_as_int' is your encoded full text as integers
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

# Create sequences of length 101 (100 + target)
seq_length = 100
sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

# Split into input and target
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

sequences = sequences.map(split_input_target)

In [27]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

data = sequences.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)